Lab02
========



## Context
#### Neural Network
+ Neural Network Review
+ Tensorboard

#### Deep Feedforward Neural Network
+ Optimization
+ Vanishing Gradient & Activation Function
+ Regularizer

##  Perceptron

+ 생물학의 신경망(뉴런)을 묘사한 모델

각 뉴런은 다른 뉴런이 출력한 결과를 입력 받아 특정 연산을 수행합니다. 계산 결과가 임계치를 넘으면 활성화되어 다음 뉴런에 값을 전달하고, 넘지 않으면 활성화되지 않아 다음 뉴런에 값을 전달하지 않습니다.

 ![Perceptron](./Images/Perceptron.png)

(input) x<sub>0</sub>, x<sub>1</sub>, x<sub>2</sub> $\cdot\cdot\cdot$: 입력되는 뉴런의 축색돌기로부터 전달되는 신호의 양

w<sub>0</sub>, w<sub>1</sub>, w<sub>2</sub> $\cdot\cdot\cdot$: 시냅스의 강도, 즉 입력되는 뉴런의 영향력을 나타냅니다.

w<sub>0</sub>x<sub>0</sub> + w<sub>1</sub>x<sub>1</sub> + w<sub>2</sub>x<sub>2</sub> $\cdot\cdot\cdot$: 입력되는 신호의 양과 해당 신호의 시냅스 강도가 곱해진 값의 합계

f : 최종 합계가 다른 뉴런에게 전달되는 신호의 양을 결정짓는 규칙, 이를 활성화 함수라고 부릅니다.


## Neural Network (NN)
Neural Newtwork(이하, 신경망)란, 여러개의 퍼셉트론으로 이루어진 모델을 이야기 합니다. <br> 
일반적으로 MLP(Multilayer Neural Perceptron)는 입력 - 은닉 - 출력 3가지 층(Layer)으로 나누어진 신경망의 기본 모델입니다. <br>
기존 하나의 퍼셉트론으로 해결하지 못했던 XOR 문제(비선형 문제)를 해결할 수 있는 모델로 소개됩니다.<br>
신경망은 대부분 지도학습의 일종으로 모델을 학습하여 문제를 해결합니다. 학습단계는 크게 2단계로 나눌 수 있습니다.<br>


#### Feedforword 단계
들어온 입력을 해당 층의 가중치(w)와 벡터 연산하고, 활성화 함수를 통과시켜 다음 층으로 전달하는 단계<br>
앞 -> 뒤 방향으로 입력(Feed)이 이동하기 때문에 Feedforword 라고 부릅니다.
![Feedforword](./Images/Feedforword.png)
#### Backpropagation 단계
출력 층에서 발생하는 오류를 최소화 하기 위해 이전의 층에 오차를 줄이는 방향으로 가중치를 전달하는 단계<br>
뒤 -> 앞 방향으로 오차를 최소화하는 역전파(Back Propagation)가 이동하기 때문에 Back Propagation 이라고 부릅니다.
![Backpropagarion](./Images/Backpropagation.png)

### 어떻게 신경망이 비선형 문제를 해결할 수 있을까?
각각의 퍼셉트론은 선형 분리만을 해결할 수 있는 모델입니다. <br> 
하지만 이런 선형 분리를 할 수 있는 모델을 여러개를 모아 비선형 분리를 수행하는 것이 신경망 입니다.<br>
![HowNNSolve](./Images/HowNNSolve.png)
아래 그림을 보면 4개의 벡터 공간을 선형 분리하는 퍼셉트론들이 하나의 비선형 공간을 분류할 수 있는 벡터 공간을 형성하는 것을 확인할 수 있습니다.<br>
직관적으로는 이해하기 어려우시겠지만, 우리가 케익을 4개의 퍼셉트론들이 분할하는 대로 잘라 가운데 부분을 남기는 것을 생각해보시면 이해가 편하실겁니다.



## Tensorboard

### 특징
Tensorboard는 Tensorflow를 사용해 생성한 모델의 구조를 그래프 형태로 살펴보고, 학습간 관찰하고자 하는 값(비용 함수, 정확도, 이미지 등등..)의 변화를 실시간으로 살펴볼 수 있습니다.<br>
<img src="./Images/Tensorboard_Scalar.png" alt="TB_Scalar" style="width: 400px"/>
<img src="./Images/Tensorboard_Graph.png" alt="TB_Graph" style="width: 400px"/>


In [ ]:
from os.path import join

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras import layers, models, optimizers, Input
from keras.utils import to_categorical
from keras.backend.tensorflow_backend import set_session
from keras import regularizers

datapath = join('data','MNIST')

In [ ]:
# Keras 백 엔드인 TensorFlow의 세션 설정을 불러옵니다.
from keras.backend.tensorflow_backend import set_session

# TensorFlow의 ConfigProto() 객체에 학습시 적용할 옵션들을 명시적으로 설정할 수 있습니다.
config = tf.ConfigProto()
# GPU 옵션으로 allow_grouth를 True로 설정합니다.
config.gpu_options.allow_growth = True 

# TensorFlow는 Session이라는 실행 단위를 가지고 있는데, 해당 Session이라는에 적용할 옵션을 담고있는 Config객체를 전달합니다.
sess = tf.Session(config=config)
# 설정한 Session이라는을 현재 프로세스에 적용합니다.
set_session(sess)  

## 1. 데이터 살펴보기
이번에 사용할 데이터는 손글씨 데이터로 유명한 MNIST 데이터 입니다.<br>
MNIST(Modified National Institute of Standards and Technology database)는 손으로 쓴 숫자들로 이루어진 대형 데이터셋 입니다.<br>
일반적으로 다양한 영상 처리 시스템을 트레이닝하기 위해 사용됩니다. 또한, 기계 학습 분야의 학습 및 검증에 널리 사용됩니다. <br>
MNIST 데이터셋은 28x28 픽셀의 흑백(Gray scale) 이미지입니다.<br>

![MNIST](./Images/MNIST.png)

data 디렉토리에 준비된 데이터와 라벨을 가져옵니다.<br>

In [ ]:
X = np.load(join(datapath, 'MNIST.npy'))
labels = np.load(join(datapath,'Label.npy'))

## 2. 간단한 전처리
MNIST 데이터는 28*28에 픽셀마다 0~255 범위의 값을 가진 행렬입니다. <br>
각 픽셀에 대해 0~1 사이의 값을 갖도록 255로 나누어줍니다.<br>
그리고 네트워크에 데이터를 입력하기 편하게 하도록 2차원 이미지를 벡터형태로 펼치겠습니다.<br>
numpy.reshape() 함수를 통해 행렬의 차원을 변경할 수 있습니다.

In [ ]:
#0~1 사이의 값으로 변경
X = X / 255

#2차원 이미지에서 1 * 784(28 x 28) 벡터로
X = X.reshape(len(X), 784)

#값이 잘 변화되었는지 확인
print('최대 : {}, 최소 : {}'.format(np.max(X), np.min(X)))

이미지들의 정답을 나타내는 label 데이터는 0~9의 값을 가지고 있습니다.<br>
해당 label 데이터도 독립적인 벡터 형태로 표현하겠습니다.

### 왜 벡터를 사용하나요
여기에서 Label 데이터를 그대로 숫자로 표현해도 되지만, 벡터로 표현한 것은 숫자 이미지라는 특성이 수치적으로 1씩 증가하는 관계가 아니기 때문에<br>
서로 독립적인 관계를 갖는 벡터로 표현해주는 것이 더 적절합니다. 이러한 기법을 One-hot encoding이라 합니다.<br>
Keras에서는 연속 정수형을 벡터형태로 변환해주는 to_categorical() 함수가 존재합니다.

In [ ]:
labels = to_categorical(labels)
print(labels[0])

데이터를 학습 데이터와, 검증 데이터로 나눕니다.

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, labels, 
                                                    test_size = 0.2,
                                                    random_state = 2019)

train_x.shape

## 3. Tensorflow로 MLP 만들어보기
얼마나 Tensorflow가 사용자 친화적이지 않은지를 보여드리고 싶어 준비했습니다.<br>
물론 낮은 수준(Low Level)까지 모델을 작성할 수 있기 때문에 더 세밀한 조절이 가능합니다.<br>

### 학습에 사용되는 파라미터

#### batch_size ?
데이터셋이 큰 경우 데이터를 한번에 학습시킬 수 없기 때문에 데이터셋을 쪼개어 학습하게 되는데<br>
쪼개어 학습하는 단위를 batch 라고 부릅니다. 예제 코드에서는 batch_size가 512이므로 동시에 512장의 이미지를 학습하는 모델이 됩니다. <br>

#### epoch ?
데이터셋 전체를 한번 학습하는 단위를 epoch(에폭, 에포크) 라고 부릅니다.<br>
예제 코드에서는 batch_size가 512이고, epochs가 10 이므로, 동시에 512장의 이미지를 학습하며, 데이터셋 전체를 10번 학습하게 됩니다.

#### learning rate?
학습률이라고도하며, 비용 함수 최적화에서 얼마나 많이 가중치를 갱신할 것인지를 설정하는 파라미터입니다.<br>
너무 작은 학습률은 천천히 수렴하는 문제가 있고 너무 큰 학습률은 진동하는 문제가 있어 적절한 학습률을 선택하는 것이 중요합니다.
![lr](./Images/lr.png)


In [ ]:
batch_size=1024
epoch=10
learning_rate = 0.0006

### 모델 확인
실제 입력층은 784개의 퍼셉트론을 가지고 있지만, 표현상의 제약으로 20개의 퍼셉트론으로 표현하였습니다.<br>
![MLP_Model](./Images/MLP.png)

### 3.1 입력 차원

#### placeholder란?
Tensorflow는 placeholder라는 것을 통해 먼저 모델에게 입력되는 Tensor의 차원과 데이터 형태를 알려주어야 합니다.<br>
정답으로 사용할 label도 placeholder로 만들어 주어야 합니다.

#### batch_size를 먼저 결정합니다.
Tensorflow에서는 모델에 들어갈 데이터의 개수(batch)와 크기(차원)를 알려주어야 하므로 batch_size를 먼저 결정합니다.<br>
Keras에서는 model.fit() 에서 매개변수로 전달합니다.

In [ ]:
# //은 나누기를 했을 때 정수를 제외한 소수 부분을 버리는 연산입니다. ex) 3/2 = 1.5, 3//2 = 1
step_size = len(train_x)//batch_size

#입력 층의 모양을 잡아줍니다.
input_shape = (batch_size, len(train_x[0]))
#출력 층의 모양을 잡아줍니다.
output_shape = (batch_size, len(train_y[0]))

# 입력으로 사용할 x와 정답으로 사용할 label의 placeholder를 생성합니다. 
x = tf.placeholder(tf.float32, shape = input_shape, name = 'input_data')
y = tf.placeholder(tf.int8, shape = output_shape, name = 'label_data')

### 3.2 Layer
네트워크에 Layer를 추가하는건 Keras의 함수형 모델과 매우 유사합니다.<br> 저희가 이번에 만들 모델은 Dense라고 불리는 모든 신경이 연결되고 연산되는 Layer입니다.<br> 아래 예제를 통해 입력 변수, 활용할 퍼셉트론 수(units), 활성화 함수를 설정하여 모델을 만들어보겠습니다.<br>

In [ ]:
# 은닉층을 추가합니다. 은닉층은 10개의 퍼셉트론과 활성화 함수로 relu를 사용합니다.
hidden = tf.layers.dense(x, units = 10, activation = tf.nn.relu)
# 출력층을 추가합니다. 촐력층은 10개의 퍼셉트론과 활성화 함수로 sigmoid를 사용합니다.
output = tf.layers.dense(hidden, units = 10, activation= tf.nn.sigmoid)

### 3.3 Optimizer
Tensorflow는 비용 함수와 optimizer를 하나 하나 직접 설정합니다. (Keras도 할 수는 있습니다.)<br>
또한, 최적화기에 비용 함수를 증가시키거나, 감소시키도록 지정할 수 있습니다.

In [ ]:
# 비용 함수를 정의합니다. softmax cross entropy의 평균을 사용합니다. 
# 오차를 산출하기 위해, 모델에서 나온 출력과 실제 정답을 매개변수로 전달합니다.
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)
)
# 출력을 softmax를 취해 각 클래스 별 확률로 뽑아냅니다.
prediction = tf.nn.softmax(output)
# AdamOptimizer를 정의합니다.
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# 비용 함수를 최소화 하는 방향으로 최적화 합니다.
train_step = optimizer.minimize(loss)

### 3.4 Accuracy
Tensorflow에서는 모델이 출력한 값과, 정답(label)을 가지고 직접 계산해야합니다.

In [ ]:
# 정확도를 계산하는 함수를 정의합니다. prediction과 label은 (512, 10) 데이터
# 모델에서 나온 결과를 tf.argmax() 함수를 사용해 가장 높은 값(확률)을 담고 있는 인덱스를 반환합니다. (512, 1)
# tf.equal() 함수를 통해 비교합니다.
correct_prediction = tf.equal(tf.argmax(prediction, axis = 1),
                            tf.argmax(y, axis = 1))
# 비교한 결과(논리값)를 float으로 형변환을 하면, True = 1, False = 0이 되어
# tf.reduce_mean() 함수로 평균을 취하면 정확도가 됩니다.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

# 여기에서 한가지 tf.reduce_mean() 함수는 이름과 다르게 평균을 구하는 함수 입니다.
# 주로 비용 함수(Loss)에 사용하는데, 일반적으로 우리가 비용 함수를 최소화 하는 방향으로
# 최적화를 진행하기 때문에 비용 함수의 평균을 최소화한다. 라는 의미를 담고 있는 것으로 보입니다.
# 하지만 실제 연산은 입력된 데이터의 평균을 구합니다.

### 3.5 Session
Tensorflow는 Session이라는 단위로 연산을 수행합니다. 학습 준비를 마치기 위해 Session을 생성합니다.<br>
Keras에서와 동일하게 GPU 메모리 제한을 적용하겠습니다.

In [ ]:
# Tensorflow는의 ConfigProto() 객체에 학습시 적용할 옵션들을 명시적으로 설정할 수 있습니다.
config = tf.ConfigProto()
# GPU 옵션으로 allow_grouth를 True로 설정합니다.
config.gpu_options.allow_growth = True 

# Tensorflow는 세션이라는 실행 단위를 가지고 있는데, 
# 해당 세션에 적용할 옵션을 담고있는 Config객체를 전달합니다.
sess = tf.Session(config=config)

### 3.6 Tensorboard
Tensorflow에서는 살펴보고자 하는 변수들을 하나하나 summary라는 곳에 등록해주고, 매 학습마다 현황을 출력해주어야합니다.

In [ ]:
# Tensorboard에서 Loss와 정확도를 확인하기 위해 summary에 등록해줍니다.
tf.summary.scalar('cross_entropy', loss)
tf.summary.scalar('accuracy', accuracy)
# 등록한 모든 summary를 합칩니다.
merge = tf.summary.merge_all()

# 학습과 검증 과정을 나누어 살펴보기 위해 2개로 나눕니다.
train_writer = tf.summary.FileWriter('./tensorflow/train', sess.graph)
test_writer = tf.summary.FileWriter('./tensorflow/test')

### 3.7 Initialize Variable
Tensorflow 모델에 들어가는 모든 가중치(weight)들은 변수(Variable) 자료형으로 되어있습니다.<br>
일반적인 프로그래밍 언어에서도 변수들의 값을 초기화 해주는 것 처럼 Tensorflow도 가중치들을 초기화 시켜준 후에 학습을 진행합니다.

In [ ]:
# 모델 내의 전역 변수들을 초기화 시켜주는 함수
init = tf.global_variables_initializer()
# Session.run을 통해 변수 초기화 함수를 수행합니다.
sess.run(init)

### 3.8 Train and Valid
Tensorflow에서는 Session.run()이라는 함수로 연산을 진행할 수 있으며, feed_dict라는 매개변수에 파이썬 딕셔너리 형태로 모델에 데이터를 입력합니다.

In [ ]:
#학습 중 변하는 비용함수 값, 정확도 등을 확인하기 위해 저장합니다.
train_loss = list()
test_loss = list()
train_acc = list()
test_acc = list()
global_step = 0

for i in range(epoch):
    for j in range(step_size):
        # 48000개의 훈련 데이터에서 batch_size만큼 비복원 추출합니다.
        # numpy.random.choice() 함수는 인덱스를 반환합니다.
        rand_index = np.random.choice(len(train_x),size = batch_size)
        rand_x = train_x[rand_index]
        rand_y = train_y[rand_index]
        
        # Session.run() 함수를 통해 학습(train_step)을 진행합니다. 
        # feed_dict에 넣을 데이터 및 라벨에 맞는 placeholder와 쌍을 이루어 입력합니다.
        sess.run(train_step, feed_dict = {x: rand_x, y: rand_y})
        
        # Session.run() 함수를 통해 Tensorboard에 기록할 정보와 Loss 및 정확도를 얻습니다.
        summary, temp_train_loss, temp_train_acc = sess.run([merge, loss, accuracy], feed_dict={x: rand_x, y: rand_y})
        
        
        # 학습 데이터에 대한 현황을 저장합니다.
        train_writer.add_summary(summary, global_step)
        global_step += 1
    
    # 12000개의 검증 데이터에서 batch_size만큼 비복원 추출합니다.
    eval_index = np.random.choice(len(test_x), size=batch_size)
    eval_x = test_x[eval_index]
    eval_y = test_y[eval_index]
    
    # Session.run() 함수를 통해 Tensorboard에 기록할 정보와 Loss 및 정확도를 얻습니다.
    summary, temp_test_loss, temp_test_acc = sess.run([merge, loss, accuracy], feed_dict={x: eval_x, y : eval_y})
    
    #학습 데이터에 대한 비용함수 결과 값과 정확도를 저장합니다.
    train_loss.append(temp_train_loss)
    train_acc.append(temp_train_acc)
    
    #검증 데이터에 대한 비용함수 결과 값과 정확도를 저장합니다.
    test_loss.append(temp_test_loss)
    test_acc.append(temp_test_acc)

    
    # 현황을 출력합니다.
    acc_and_loss = [(i + 1), temp_train_loss, temp_train_acc, temp_test_loss,
                        temp_test_acc]
    
    print('Epoch # {}. Train Loss: {:.5f}, Train Acc : {:.3f}, Test Loss : {:.5f} , Test Acc : {:.3f}'.format(*acc_and_loss))
    # 검증 데이터에 대한 현황을 Tensorboard에 저장합니다.
    test_writer.add_summary(summary, global_step)

### 3.9 Print
위의 학습과정중 저장한 비용함수, 정확도를 그래프로 출력하여 어떤 과정을 거쳐 학습되었는지 확인해보겠습니다.

In [ ]:
#각 epoch마다의 정확도를 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(train_acc)
plt.plot(test_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(train_loss,)
plt.plot(test_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## 4. Keras로 MLP 만들어보기
Tensorflow에 비해 얼마나 사용자 친화적인지 비교해보면서 진행하겠습니다.


### 4.1 입력, 출력 차원

#### for Keras
Keras에서는 별도의 placeholder를 생성하지 않고 입력변수의 크기(차원)만 알려주어도 됩니다.
이번 실습에서 사용할 데이터인 MNIST 데이터는 28 * 28 의 2D이미지입니다 이를 784(28x28) * 1 1차원 벡터로 변환하여 사용하므로 아래와 같이 정의하였습니다.<br>

Keras에서는 layer를 출력 차원으로 설정해 줍니다. 따라서 별도의 출력 차원을 만들지 않고 모델을 만들겠습니다.

#### for Tensorklow
Tensorflow에서는 placeholder라는 것을 통해 먼저 모델에게 입력되는 Tensor의 차원과 데이터 형태를 알려주어야 합니다.<br>
keras와 달리 출력 차원 또한 placeholder로 생성하여야 합니다.

In [ ]:
# 입력층은 MNIST의 입력인 784개의 차원(퍼셉트론 수)을 사용합니다.
input_shape = (784,)

input_tensor = Input(input_shape, name = 'input')

### 4.2 Layer

#### for Keras
Keras가 사용하는 함수형 모델은 Tensorflow와 매우 유사합니다. 큰 차이점이 없습니다.

In [ ]:
# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 10, activation = 'relu', name = 'hidden')(input_tensor)
# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

#입력차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

### 4.3 Optimizer

#### for Keras
Keras에서는 모델을 만들고, model.compile(optimizer = '', loss = '' , metrics=[]) 함수를 통해 손쉽게 최적화기와 비용 함수를 설정했습니다.

#### for Tensorflow
Tensorflow는 비용 함수와 optimizer를 하나 하나 직접 설정합니다. (Keras도 할 수는 있습니다.)<br>
또한, 최적화기에 비용 함수를 증가시키거나, 감소시키도록 지정할 수 있습니다.

In [ ]:
#10개의 숫자 클래스를 분류하는 것이므로 categorical_crossentropy를 비용함수로 사용합니다.
#loss를 줄여주는 Optimizer는 Adam을 직접 넣어 사용할 수도 있습니다.
model.compile(optimizer=keras.optimizers.Adam(lr = learning_rate),
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

### 4.4 Accuracy
#### for Keras
Keras에서는 model.fit() 함수로 데이터를 넣음과 동시에 학습을 시작하며, 알아서 훈련 데이터와 검증 데이터에 대해 정확도가 출력되었습니다.

#### for Tensorflow
Tensorflow에서는 모델이 출력한 값과, 정답(label)을 가지고 직접 계산해야합니다.

### 모델 확인

#### for Keras
Keras는 model.summary() 함수를 통해 모델의 각 층을 확인할 수 있습니다.

#### for Tensorflow
Tensorflow는 별도의 모델 확인 함수를 제공하지 않으며 다만 Tensorboard를 통해 확인할 수 있습니다.

In [ ]:
model.summary()

### 4.6 Tensorboard
#### for Keras
Keras에서는 콜백함수를 사용해 손쉽게 Tensorboard를 사용할 수 있었습니다.
#### for Tensorflow
하지만, Tensorflow에서는 살펴보고자 하는 변수들을 하나하나 summary라는 곳에 등록해주고, 매 학습마다 현황을 출력해주어야합니다.


### callback ?
함수의 실행 시점을 프로그래머가 결정하는 것이 아닌, 시스템이 결정하여 실생하는 함수를 말합니다.<br>
이번 실습에서는 epoch이 끝날때마다 모델이 학습되는 기록(log)을 저장하는 콜백 함수를 사용하도록 하겠습니다.

In [ ]:
# Keras 의 callback 모듈 중 Tensorboard에 사용할 수 있는 log event 파일을 저장할 수 있는 콜백 함수를 가져옵니다.
tb_hist = keras.callbacks.TensorBoard(log_dir='./logdir/keras_logdir', 
                                      write_graph=True)

### 4.7 Initialize Variable

#### for keras
저희 예제에서는 사용하지 않지만 Layer를 추가할 때 입력 인자중 kernel_initializer를 조작하여 설정할 수 있습니다.<br>
#### for Tensorflow 
Tensorflow 모델에 들어가는 모든 가중치(weight)들은 변수(Variable) 자료형으로 되어있습니다.<br>
일반적인 프로그래밍 언어에서도 변수들의 값을 초기화 해주는 것 처럼 Tensorflow도 가중치들을 초기화 시켜준 후에 학습을 진행합니다.

### 4.8 Train and Valid
#### for Keras
model.fit() 함수를 사용해 모델을 학습시킬 수 있습니다.<br>

#### for Tensorflow
Tensorflow에서는 Session.run()이라는 함수로 연산을 진행할 수 있으며, feed_dict라는 파이썬 딕셔너리를 이용해 모델에 데이터를 입력합니다.

In [ ]:
#학습 중 변화하는 주요지표들을 다음에 확인하기 위해 history_Adam라는 변수를 선언합니다.
#위에서 만든 model에 학습용 입력/정답 데이터, 검증용 입력/정답 데이터, batch_size, epoch, callback함수를 설정하여 학습을 수행합니다.
history_Adam = model.fit(train_x, train_y, 
          validation_data = [test_x, test_y],
          batch_size=batch_size,
          epochs=epoch,
          verbose = 2,
          callbacks=[tb_hist])

### 4.9 Print
위의 학습과정중 저장한 비용함수, 정확도를 그래프로 출력하여 어떤 과정을 거쳐 학습되었는지 확인해보겠습니다.

#### for Keras
Model의 fit을 통하여 학습을 수행할 때 주요 지표들을 반환하여 줍니다. 위의 코드의 history가 그 반환되는 값들을 저장하는 변수입니다.

#### for Tensorflow
저장하고자하는 주요 지표들을 직접 변수를 선언하여 학습 과정중 저장하여 줍니다.

In [ ]:
#각 epoch마다의 정확도를 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_Adam.history['acc'])
plt.plot(history_Adam.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_Adam.history['loss'])
plt.plot(history_Adam.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Optimization
신경망 알고리즘의 최적화(Optimization)는 **비용함수**의 결과 값이 비용함수의 **최솟값**이 되도록 Weight를 조절하는 것을 의미합니다.

### 비용함수
비용함수는 이 머신러닝 모델을 수용했을 때 감안해야할 것들을 숫자로 표시하는 함수입니다. 기본적으로는 오차율(분류 문제의 경우 오차율, 추론 문제의 경우 실제 값과의 오차)을 가장 중요하게 세우며 모델의 복잡도나 경계값과의 거리(마진) 등을 포함하기도 합니다.

+ Mean Squared Error: 실제 관측값과 추론 값의 차이를 제곱하여 평균을 구하는 비용함수
+ Categorical Crossentropy: 신경망의 출력에 log를 취하고 -1을 곱한 값들을 더하는 비용함수인 Crossentropy에 여러 클래스에 대한 출력 중 정답 클래스로 예측한 확률만을 Crossentropy 비용함수 값을 구하여 더하는 비용함수

#### 그럼 최적화에서 말하는 Convex란 무엇일까?
Convex는 아래로 볼록한 함수 형태를 뜻합니다. Mean Square Error의 경우 다음과 같은 식으로 표현됩니다. <br><br>
 $$MSE = \sum_{i=1}^m \frac{1}{m}{(\hat{y_i}-y_i)^2}$$


MSE는 내부에 $(\hat{y_i}-y_i)^2$ 항을 가지고 있기 때문에 Convex한 형태의 비용 함수(Loss)가 됩니다. <br>
Convex하다는 말은 최솟값이 반드시 존재한다는 의미이며, 이는 비용 함수 즉 오차의 최솟값을 구할 수 있다는 의미입니다. <br>
아래로 볼록한 형태를 가진 함수의 최솟값은 해당 함수의 미분 값이 0이되는 지점을 찾으면 됩니다.<br>

![title](Images/Second_Order_func.png)
 
#### 비용 함수가 Convex 하지만, 미분 값이 0이 되는 지점을 여러 개 가지고 있다면?
우리는 기울기가 0인 지점을 직접 계산하여 찾을 수 있지만, 식이 너무 복잡해지는 경우 찾을 수 없게 됩니다.<br>
또한, 컴퓨터는 기울기가 0인 지점을 한번에 계산하는 것이 아닌 기울기를 따라 이동하며 찾아냅니다.<br>
따라서 우리가 찾고자 하는 최솟값이 최적해(Global Optimum)가 아닐 수 있습니다. 이를 지역해(Local Optimum)라고 합니다.

![title](Images/Fourth_Order_func.png)

### 경사하강법(Gradient Descent)
**모든 공간**상의 **지역 최적해**를 찾을 수 있는 방법으로 현재 어떤 경사면에 있는지에 따라 입력값을 조절하여 극값에 해당하는 입력값을 찾는 방법입니다.<br> 시작 입력값에 따라 결과값이 달라지며 모든 공간에 적용 가능하다는 범용성과 반복할 수록 최적해에 근접한다는 특징이 있어 모든 최적화기의 기본이 되는 방법입니다.

+ **Stochastic Gradient Descent(SGD)**: 경사하강법의 지역최적해 문제와 데이터가 많을 때 연산량이 커지는 문제를 해결하기 위해 고안된 것으로 데이터의 일부분을 사용하여 Gradient Descent 과정을 여러번 거듭하고 그 중 최적의 해로 뽑힌 값을 취하는 방법입니다.

+ **Momentum**: SGD에 관성 개념을 도입하여 이전에 이동한 정도를 기억하고 이를 다음 이동에 적용하는 방법입니다. 이는 SGD가 진동에 빠졌을 때(이동 방향이 계속 달라지는 경우) 연산 수를 줄일 목적으로 고안됐습니다.

+ **Adagrad**: 이전에 변화한 횟수가 적을 수록 더 많이씩 변화하고 변화 횟수가 많을 수록 더 조금씩 변화하도록 설계된 경사하강법입니다. 대체적으로 변화 횟수가 적으면 아직 최적해와 거리가 멀 확률이 높기 때문에 더 많이 변화하고 변화 횟수가 많으면 거리보다는 세밀한 값조정이 필요할 확률이 높기 때문에 더 적게 변화하도록 고안됐습니다.

+ **RMSProp**: 위에서 설명드린 Adagrad는 변화 횟수가 많을 수록 Learning Rate를 0으로 수렴시키기 때문에 계속 수행하다보면 변화량이 0이 됩니다. 이런 문제를 개선하기 위해 고안된 방법으로 Learning Rate를 무한정 줄이는 것이 아니라 지수평균을 이용하여 변수간 상대적 변화량 크기 차이는 유지하는 방법입니다.

+ **Adam**: RMSProp과 Momentum을 결합한 방법으로 현재까지 알려진 최적화기 중 최고의 성능을 보이는 것으로 알려져있습니다.


![title](Images/Optimizer.gif)
![title](Images/Optimizer2.gif)

### 가중치 값의 변화를 관찰해보는 예제입니다.
tensorflow로 작성되었지만, 최대한 간결하게 작성했습니다.

In [ ]:
# 입력으로 사용할 데이터 입니다.
x_train = [1, 2, 3]
y_train = [1, 2, 3]

In [ ]:
# y = aw + b 식을 만들기 위한 변수 입니다.
w = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random_normal([1]))

In [ ]:
# y = aw + b
y_hat = x_train * w + b

# 비용 함수는 회귀에서 배운 Mean Square Error 입니다. (잔차의 제곱 평균)
loss = tf.reduce_mean(tf.square(y_hat - y_train))

# 최적화기는 Gradient Descent Optimizer 사용합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
# 비용 함수를 최소화 하는 방향으로 최적화합니다.
train_op = optimizer.minimize(loss)

In [ ]:
# 모든 변수를 초기화 합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for step in range(100):
    # 학습을 진행합니다.
    sess.run(train_op)
    if step % 20 == 0 :
        print('step {}\tloss: {:.5f}, w : {},\tb : {}'.format(step, sess.run(loss), sess.run(w), sess.run(b)))

## 5. 최적화기 변경해보기
다른 변수는 기존의 모델을 유지하고 최적화기를 변경하여 비교해보겠습니다.

### 5.1 Stochastic Gradient Descent(SGD)
첫번째로 사용할 최적화기는 Stochastic Gradient Descent(SGD)입니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 10, activation = 'relu', name = 'hidden')(input_tensor)

# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

# 입력 차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'SGD',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_SGD = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

### 5.2 Momentum
두번째로 사용할 최적화기는 Momentum으로 Keras에서는 Momentum을 SGD의 입력 변수를 통하여 사용할 수 있습니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 10, activation = 'relu', name = 'hidden')(input_tensor)

# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

# 입력 차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = keras.optimizers.SGD(momentum = 0.7),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_Momentum = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

### 5.3 Adagrad
세번째 사용할 최적화기는 Adagrad입니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 10, activation = 'relu', name = 'hidden')(input_tensor)

# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

#입력차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'Adagrad',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_Adagrad = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

### 5.4 RMSProp
네번째 사용할 최적화기는 Adagrad의 변형인 RMSProp입니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 10, activation = 'relu', name = 'hidden')(input_tensor)

# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

#입력차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'RMSProp',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_RMSProp = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

위에서 구한 여러 지표들을 그래프로 나타내어 보겠습니다.

In [ ]:
#각 epoch마다의 정확도를 최적화기 종류에 따라 분리하여 그래프로 나타냅니다
plt.plot(history_Adam.history['val_acc'])
plt.plot(history_SGD.history['val_acc'])
plt.plot(history_Momentum.history['val_acc'])
plt.plot(history_RMSProp.history['val_acc'])
plt.plot(history_Adagrad.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Adam', 'SGD', 'Momentum' , 'RMSProp', 'Adagrad'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 최적화기 종류에 따라 분리하여 그래프로 나타냅니다
plt.plot(history_Adam.history['val_loss'])
plt.plot(history_SGD.history['val_loss'])
plt.plot(history_Momentum.history['val_loss'])
plt.plot(history_RMSProp.history['val_loss'])
plt.plot(history_Adagrad.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Adam', 'SGD', 'Momentum' , 'RMSProp', 'Adagrad'], loc='upper left')
plt.show()

## 활성화 함수
앞서 만든 모델을 살펴보면 각 층마다 activation이라는 것을 지정해줍니다. 이것은 퍼셉트론과 퍼셉트론의 집합이라고 볼 수 있는 신경망의 결과로 나온 입력신호의 총합을 우리가 얻고자하는 출력형태로 변환해주는 역할을 합니다.

+ **Step**: 입력 신호의 값에 따라 0과 1을 반환하도록 설계된 함수입니다. 0과 1이 나뉘는 임계값은 보통 0을 기준으로 하며 사용자에 따라 따로 지정하기도 합니다.

+ **Sigmoid**: 로지스틱 회귀에서 회귀식으로 분류문제를 해결하기 위해 고안된 함수로 입력신호의 총합을 0~1사이의 실수값으로 반환하도록 설계된 함수입니다.

+ **Tanh**: 쌍곡사인, 쌍곡코사인에 의해 만들어진 쌍곡선 함수로 모든 값을 -1 ~ 1 사이의 값으로 변환하여 활성화 함수의 일종으로 사용됩니다.

+ **Rectified Linear Unit(ReLU)**: Sigmoid의 **Vanishing Gradient Problem**을 문제삼아 이러한 문제를 근본적으로 해결하고자 고안된 함수로 입력신호의 총합이 0보다 작으면 0을 0보다 크다면 그 입력신호의 총합 그대로 반환하도록 설계된 함수입니다.

+ **Softmax**: 로지스틱 회귀에서 사용한 Sigmoid 함수를 다중 분류 문제(Multi-Class Problem)를 해결할 수 있도록 고안된 것으로 클래스 수 만큼의 출력신호들을 총합이 1이 되는 값들로 반환하도록 설계된 함수입니다.

![activation](Images/activation.png)

## Vanishing Gradient Problem
신경망은 역전파를 통해 모델을 학습 데이터에 맞게 수정합니다. 이 역전파 과정은 미분 값을 체인룰로 곱하며 진행됩니다. 그리고 Sigmoid 활성화 함수 특성상 미분값의 최대 값이 0.25이기 때문에 신경망의 역전파 과정중 모델의 깊이가 깊을수록 감소시켜야할 Gradient항이 없어져 더 이상 학습이 진행되지 않는 문제입니다. 이 문제는 넓고 깊은 신경망을 시간을 들여 충분히 반복시키면 모든 문제를 해결할 수 있을거라는 기계학습 분야에 암흑기를 가져오게 했던 문제입니다.


## 6. Vanishing Gradient 확인하기
Vanishing Gradient Problem이 발생하는 활성화 함수인 Sigmoid 활성화 함수를 사용한 층을 다수 만들어서 확인해보겠습니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 총 10개의 은닉층 모두 sigmoid 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 700, activation = 'sigmoid', name = 'hidden')(input_tensor)
hidden = layers.Dense(units = 600, activation = 'sigmoid', name = 'hidden2')(hidden)
hidden = layers.Dense(units = 500, activation = 'sigmoid', name = 'hidden3')(hidden)
hidden = layers.Dense(units = 400, activation = 'sigmoid', name = 'hidden4')(hidden)
hidden = layers.Dense(units = 300, activation = 'sigmoid', name = 'hidden5')(hidden)
hidden = layers.Dense(units = 200, activation = 'sigmoid', name = 'hidden6')(hidden)
hidden = layers.Dense(units = 100, activation = 'sigmoid', name = 'hidden7')(hidden)
hidden = layers.Dense(units = 50, activation = 'sigmoid', name = 'hidden8')(hidden)
hidden = layers.Dense(units = 10, activation = 'sigmoid', name = 'hidden9')(hidden)

# 출력 층
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)
# 모델 정의
model = models.Model(input_tensor, output)
# 모델 생성(컴파일), 최적화기는 Adam 비용 함수는 범주형 분류이므로 categorical crossentropy를 사용합니다.
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_Sigmoid = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

In [ ]:
#각 epoch마다의 정확도를 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_Sigmoid.history['acc'])
plt.plot(history_Sigmoid.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_Sigmoid.history['loss'])
plt.plot(history_Sigmoid.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

이번엔 활성화 함수로 sigmoid 대신 relu를 사용해보겠습니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 총 10개의 은닉층 모두 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 700, activation = 'relu', name = 'hidden')(input_tensor)
hidden = layers.Dense(units = 600, activation = 'relu', name = 'hidden2')(hidden)
hidden = layers.Dense(units = 500, activation = 'relu', name = 'hidden3')(hidden)
hidden = layers.Dense(units = 400, activation = 'relu', name = 'hidden4')(hidden)
hidden = layers.Dense(units = 300, activation = 'relu', name = 'hidden5')(hidden)
hidden = layers.Dense(units = 200, activation = 'relu', name = 'hidden6')(hidden)
hidden = layers.Dense(units = 100, activation = 'relu', name = 'hidden7')(hidden)
hidden = layers.Dense(units = 50, activation = 'relu', name = 'hidden8')(hidden)
hidden = layers.Dense(units = 10, activation = 'relu', name = 'hidden9')(hidden)

output = layers.Dense(units = 10,
                      activation = 'softmax',
                      name = 'output')(hidden)

model = models.Model(input_tensor, output)

model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_Relu = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

위에서 구한 여러 지표들을 그래프로 나타내어 보겠습니다.

In [ ]:
#각 epoch마다의 정확도를 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_Relu.history['acc'])
plt.plot(history_Relu.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_Relu.history['loss'])
plt.plot(history_Relu.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## 정규화
우리가 앞서 배우고 적용한 최적화기는 학습데이터에 대해 학습학고 비용함수를 최소화하는 최적화입니다. 즉 흔히 말하는 과적합(Overfitting)문제에 빠지게 됩니다. 하지만 기계학습의 최종목표는 학습데이터 뿐만 아니라 앞으로 들어올 새 데이터에도 최적화된 모델이기 때문에 정규화 과정이 필요합니다.

### 파라미터 정규화
모델이 복잡할 수록 과적합되기 쉽다는 점을 이용하여 과적합 문제를 해결하기 위해 우리는 각 파라미터를 하나의 비용으로 생각하여 이를 최소화하는 방향으로 정규화를 수행할 수 있습니다. 특정 파라미터가 0이 된다면 모델의 복잡도가 한 차원 감소하게 되므로 모델의 복잡도를 최소화 할 수 있습니다.

+ **L2 파라미터**
파라미터 정규화 방법으로 각 파라미터의 제곱값을 페널티로 삼는 방법입니다. 파라미터들의 제곱을 더한 후 데이터의 수로 나눈 것을 기본으로 여기에 사용자가 설정하는 하이퍼파라미터(보통 0.01)를 통해 이 정규화 항에 얼마나 가중치를 둘 것인지 설정하여 사용합니다.

+ **L1 파라미터**
파라미터 정규화 방법으로 각 파라미터의 절대값을 페널티로 삼는 방법입니다. 마찬가지로 파라미터들의 절대값을 더한 후 데이터의 수로 나눈것을 기본으로 여기에 사용자가 설정하는 하이퍼파라미터(보통 0.01)를 통해 이 정규화 항에 얼마나 가중치를 둘 것인지 설정하여 사용합니다.

### 데이터셋 확장
우리가 모든 데이터(수집된 데이터 + 수집 될 데이터)를 학습데이터로 가질 수는 없지만 **큰 수의 법칙**에 따라 학습데이터가 충분히 커지면 모든 데이터에서의 결과와 가까울 가능성이 높다는 것을 아이디어로 고안된 정규화 방법으로 주로 이미지를 활용한 분류문제에 적용될만한 방법입니다. 주로 가진 데이터를 대칭이동, 노이즈 추가, 늘이기/줄이기, 회전, 이동, 색상변환 등을 통해 이루어집니다.

![augmentation](Images/augmentation.png)

### Batch Normalizaion
배치 정규화는 활성화함수의 활성화값 또는 출력값을 정규화(정규분포로 만든다)하는 작업을 말합니다. 신경망의 각 layer에서 데이터(배치)의 분포를 정규화합니다.<br>
일종의 노이즈를 추가하는 방법으로 이는 배치마다 정규화를 함으로써 전체 데이터에 대한 평균의 분산과 값이 달라질 수 있습니다.<br>

* 학습 속도가 개선됩니다. 
* 가중치 초깃값 선택의 의존성이 적어집니다. 
* 과적합(overfitting) 위험을 줄일 수 있습니다. 
* Gradient Vanishing 문제 해결

### 배깅(bootstrap aggregating)
특징 전체를 이용하는 것이 아닌 일부를 활용한 여러 모델을 생성한 후 이들을 종합하는 일종의 앙상블 방법입니다. 특징을 모두 사용하는 것이 아니기 때문에 모델이 단순화될 가능성이 높아 정규화 방법으로 사용합니다.

### Dropout
신경을 꺼버리면 해당 신경은 학습에 영향을 미치지 않게 됩니다. 무작위로 선택된 신경이 꺼진 채 학습된 모델을 여럿 생성하여 종합하는 앙상블 방법이 Dropout입니다. 이러한 Dropout방법이 정규화 과정으로 볼 수 있는 이유는 신경망 구조는 학습 과정 중 동조화 현상을 겪게 되는데 이를 통해 활성화 되지 않아야할 신호가 활성화 되기도 합니다. Dropout은 이런 동조화 현상을 완화시켜 활성화 기준을 더 엄격하게 하고 필요치 않은 특징을 더 쉽게 배제할 수 있도록 하여 모델을 단순화시키고 정규화의 과정으로 볼 수 있게됩니다.

![dropout](Images/dropout.png)

## 7. 정규화 사용해보기

### 7.1 L1 Parameter 추가
Keras에서는 Layer의 kernel_regularizer인자를 조절하여 L1, L2 파라미터 정규화를 추가할 수 있습니다.

In [ ]:
epoch = 10

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수를 사용합니다.
hidden = layers.Dense(units = 10, kernel_regularizer = regularizers.l1_l2(l1 = 0.1, l2 = 0.00), activation = 'relu', name = 'hidden')(input_tensor)
# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

#입력차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_l1 = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

### 7.2 L2 Parameter 추가

In [ ]:
# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수와 L1,L2 정규화를 사용합니다.
hidden = layers.Dense(units = 10, kernel_regularizer = regularizers.l1_l2(l1 = 0.00, l2 = 10.0), activation = 'relu', name = 'hidden')(input_tensor)
# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

#입력차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_l2 = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

### 7.3 비교군 생성
비교군을 만들기 위해 L1, L2 모두 0을 넣은 모델을 하나 생성합니다.

In [ ]:
# 은닉층은 10개의 퍼셉트론과 relu 활성화 함수와 L1,L2 정규화를 사용합니다.
hidden = layers.Dense(units = 10, kernel_regularizer = regularizers.l1_l2(l1 = 0.00, l2 = 0.00), activation = 'relu', name = 'hidden')(input_tensor)
# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10, activation = 'softmax', name = 'output')(hidden)

#입력차원과 출력층을 인자로 모델을 정의합니다.
model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_zero = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

### 7.4 Dropout 사용해보기
Keras에서는 이때까지 사용하던 Dense Layer대신 Dropout Layer를 사용하여 Dropout 정규화를 사용할 수 있습니다. 이 때 퍼셉트론의 수를 의미하던 units대신 rate를 넣어 해당 Layer의 입력에서 어느정도의 비율로 Dropout을 수행할지 결정할 수 있습니다.

In [ ]:
# 입력층
input_shape = (784,)
input_tensor = Input(input_shape)

# 은닉층은 입력층을 80% 비율의 Dropout을 적용한 층을 사용합니다.
hidden = layers.Dropout(rate = 0.8,
                     name = 'hidden')(input_tensor)

# 출력층은 우리가 예측할 0~9의 숫자 개수인 10개의 퍼셉트론과 softmax 활성화 함수를 사용합니다.
output = layers.Dense(units = 10,
                      activation = 'softmax',
                      name = 'output')(hidden)

model = models.Model(input_tensor, output)

In [ ]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_Dropout = model.fit(train_x, train_y,
                   validation_data = [test_x, test_y],
                   batch_size = batch_size,
                   epochs = epoch,
                   verbose = 2)

In [ ]:
#각 epoch마다의 정확도를 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_l1.history['acc'])
plt.plot(history_Dropout.history['acc'])
plt.plot(history_l2.history['acc'])
plt.plot(history_zero.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['L1', 'Dropout', 'L2', 'Zero'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_l1.history['loss'])
plt.plot(history_Dropout.history['loss'])
plt.plot(history_l2.history['loss'])
plt.plot(history_zero.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['L1', 'Dropout', 'L2', 'Zero'], loc='upper left')
plt.show()

In [ ]:
#각 epoch마다의 정확도를 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_l1.history['val_acc'])
plt.plot(history_Dropout.history['val_acc'])
plt.plot(history_l2.history['val_acc'])
plt.plot(history_zero.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['L1', 'Dropout', 'L2', 'Zero'], loc='upper left')
plt.show()

#각 epoch마다의 비용함수 값을 학습용 데이터, 검증용 데이터 분리하여 그래프로 나타냅니다
plt.plot(history_l1.history['val_loss'])
plt.plot(history_Dropout.history['val_loss'])
plt.plot(history_l2.history['val_loss'])
plt.plot(history_zero.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['L1', 'Dropout', 'L2', 'Zero'], loc='upper left')
plt.show()

## 8. Tensorboard 사용해보기
MLP를 Keras와 Tensorflow로 구현해보고 Tensorboard를 사용하기 위한 준비를 마쳤습니다.<br>
그럼 이제 한번 살펴보도록 하겠습니다.

#### 실행명령어
Tensorboard는 터미널(cmd, 커맨드 등등..)에서 tensorboard 명령어를 사용해 실행할 수 있습니다. 실행 명령어는 다음과 같습니다.<br>
**tensorboard --logdir=절대경로**<br>
저 컴퓨터 기준으로 TensorBoard 1.10.0 at http://Kyles-MacBook-Pro.local:6006 (Press CTRL+C to quit)라고 출력되고 정상적으로 실행됐습니다.<br>
웹 페이지상에 localhost:6006이라고 치시면, 접속할 수 있습니다.

 <img src="./Images/Tensorboard_Command.png" alt="Tensorboard_Command" style="width: 400px"/>

#### Tab
Tensorboard에는 여러가지 탭이 있습니다. 그 중 몇가지만 살펴보도록 하겠습니다.

#### Scalar
Scalar는 모델을 학습할 때 추가했던 Loss, Accuracy 등이 저장됩니다.<br>
해당 값들은 그래프로 표현되고, 저장된 Log파일이 여러개인 경우 중첩해서 확인할 수 있습니다. Ex) 학습, 검증 양상을 같이 보고 싶을 때<br>

#### Graphs
Graphs는 모델의 형태를 살펴볼 수 있습니다. Tensorflow는 이름 그대로 Tensor라는 데이터가 흐르는(flow) 그래프 구조의 모델로 이루어져 있습니다.<br>
입력 데이터가 모델에 들어와 어디로 이동하는지를 확인할 수 있습니다.

#### Images
Images는 말 그대로 학습 진행간 저장한 이미지들을 살펴볼 수 있습니다. 예를들어 이미지의 해상도를 올리는 모델이라면, 학습이 진행됨에 따라 이미지의 해상도 변화를 살펴보는 방식으로 사용됩니다.

#### Audio
Audio는 모델이 학습 진행간 저장한 소리 파일들을 살펴볼 수 있습니다.

#### Histogram
Histogram은 학습 진행간 변화하는 데이터의 히스토그램을 그려 분포의 이동, 또는 변화를 살펴볼 수 있습니다.